In [1]:
import pandas as pd
import numpy as np
import time
import funcoes
from datetime import datetime, timedelta
from dateutil import parser
# configs
pd.set_option("display.date_dayfirst", True)
# variaveis globais
tks = ['HihrPwM6iplpMc20YafyI0X20hYBwMJjn9O5b3uRXLc%3D','NHvW6AeyLM3HFKp1HwXQIbDbGVdDOyMhvbyHgEoZY57S4pEFqdDDKy1R%2FVQHplHm','UVzI3W1fv8mAK28sWaTc%2BOlAefhF1q26WWTfaOmb%2BSY6F1b2IE43cgRMrafYMySh','oAlfna7l2QNiS6nZo4oIQSQCpNo5XX9iydYqEGfMVkDVz8fBG2Vl6bW9lChTtGtP', 'vcMSbDWqyMNu8fcVLol8l0dZ0DZhinIFKyydHCBSAb%2B5RloWjteXNY9345MIcbUQ']
metodo = 'GET'
url = "https://elysia.zeev.it"
tipoUrl = "/api/2/assignments"
filtro = {'flowId': 0}

# datas
today = datetime.today().strftime('%Y-%m-%d')
periodo1 = datetime.today() - timedelta(days=7)
periodo1 = periodo1.strftime('%Y-%m-%d')
periodo2 = today 
print(today)
print("No período de {0} e {1}".format(str(periodo1), str(periodo2)))

2022-11-04
No período de 2022-10-28 e 2022-11-04


In [3]:
# Se a API alterar seu campos novamente, sera necessario alterar apenas alguns filtros desta funcao
# sem gerar problemas maiores para refatorar este codigo
def request_Orquestra(method='GET',urlReq="https://elysia.zeev.it",typeReq="/api/2/assignments",tokens={},filters={}):
        retorno = []
        try:
                # filtros base para o payload
                if not typeReq.lower().__contains__('report'):
                        filters['recordsPerPage'] = 100
                        seconds = 1
                else: 
                        print("---- >> Requisicao lenta... << ---- ")
                        filters['recordsPerPage'] = 30
                        seconds = 3
                
                filters['mobileEnabledOnly'] = False
                # variaveis de controle
                row = 1
                i = 1
                # faz a busca para cada token
                for tk in tokens:
                        # reseta variaveis de controle
                        row = 1
                        i = 1
                        # tokens de acesso dos usuarios que queremos buscar as atribuicoes
                        authorization= {'Authorization': "Bearer " + tk}
                        while row != 0:
                                # aguardo 1s devido ao limite de requisicoes por segundo
                                time.sleep(seconds) 
                                
                                # incremento de pagina
                                filters['pageNumber'] = i
                                
                                # requisicao
                                getReq = funcoes.requests_API_Orquestra(metodo=method,urlAcesso=urlReq,tipoAcesso=typeReq,head=authorization,payload=filters)
                                
                                # resultado json
                                result = getReq.json()
                                        
                                # checa se esta vazio o resultado, o que significa que nao tem mais paginas
                                if (len(result) == 0) or (getReq.status_code != 200):
                                        # log
                                        print(result)
                                        break
                                else:
                                        # coloca cada retorno na lista
                                        retorno.append(result)
                                        i+=1
                # cada usuario do token fica em uma uma sublista da lista de retorno
                # por isto a necessidade de explodir ela dentro de outra lista
                retornoLista = [item for sublista in retorno for item in sublista]
                # log
                return retornoLista 
        except Exception as e:
                # log
                funcoes.printError(e)
                return [] 

In [58]:
# funcao que pega o retorno verifica se eh valido, caso nao seja retornar erro, se for valido formata as datas e começa a contagem de cada 
# verifica se o retorno eh uma lista 
def formata_DataFrame(lista):
    # verifica se houve retorno
    try:
        # ocorreu erro ou nao ocorreram retornos - provisorio
        if len(lista) <= 0:
            return -1
        # houve retorno de linhas
        elif len(lista) > 0:
            df = pd.DataFrame(lista)
            # Colocando o idTask para conferencia
            df['idTask'] = pd.DataFrame(df['instance'].apply(pd.Series))['id']
            # selecionando apenas as colunas necessarias
            df = df[['idTask','id','taskName','requestName','startDateTime','expirationDateTime','flow','instance']]

            # convertendo colunas para datetime
            df['expirationDateTime'] = pd.to_datetime(df['expirationDateTime'])
            df['startDateTime'] = pd.to_datetime(df['startDateTime'])

            # formatando data em DD/MM/YY
            df.loc[:,'expirationDateTime'] = df.loc[:,'expirationDateTime'].dt.strftime('%Y-%m-%d')
            df.loc[:,'startDateTime'] = df.loc[:,'startDateTime'].dt.strftime('%Y-%m-%d')

            # convertendo colunas para datetime
            df['expirationDateTime'] = pd.to_datetime(df['expirationDateTime'])
            df['startDateTime'] = pd.to_datetime(df['startDateTime'])
            
            # remove duplicatas de tarefas
            print('Contagem de tarefas \"ANTES\" da remocao das tarefas duplicatas: {0}'.format(str(len(df))))
            #df = df.drop_duplicates(subset=['idTask'])
            print('Contagem de tarefas \"DEPOIS\" da remocao das tarefas duplicatas: {0}'.format(str(len(df))))

            # adiciona coluna de tarefa atrasada
            df['taskAtrasada'] = False
            df.loc[today > df['expirationDateTime'],['taskAtrasada']] = True
            
            # tudo ok, retorna o data frame
            return df
    except Exception as e:
        funcoes.printError(e)
        return -1

In [66]:
def cont_Instancias_Orquestra(nomeSolicitacao=str, nomeTarefa=str,per1=datetime,per2=datetime,dataFrame=pd.DataFrame, status='endDateTime'):
    '''
     Funcao
     ------
            Retorna a contagem de tarefas fechadas para determinada solicitacao.

    Parameters
    ----------
        nomeSolicitacao: str (obrigatorio)
            Nome da solicitacao que sera buscada e contada a instancia.
        
        nomeTarefa: str (obrigatorio)
            Nome da tarefa para ser contada a sua quantidade.

        dataFrame: dataFrame Pandas (obrigatorio)
            Dataframe que sera utilizado como base de dados para a contagem.
        
        per1: datetime (obrigatorio)
            Periodo inicial para busca.

        per2: datetime (obrigatorio)
            Periodo final para busca.
    Retorno
    -------
        int: Retorna a quantidade de instancias ja finalizadas desta tarefa.
        * Se erro, retorna -1   
    '''
    try:
        # Formatar parametros
        nomeSolicitacao = str(nomeSolicitacao).lower().replace(" ","")
        nomeTarefa = str(nomeTarefa).lower()
        # Projetos 
        projetos = dataFrame.loc[(dataFrame['requestName'].str.lower().str.replace(" ","").str.contains(nomeSolicitacao))]
        # Contagem
        cont=0 
        for row in projetos.values:
            for instance in row[19]:
                task = str(instance['task']['name']) 
                date = parser.parse(instance[status]).strftime("%Y-%m-%d")
                if task.lower().__contains__(nomeTarefa):
                    if date >= per1 and date <= per2:
                        cont+=1
                        break
        return cont
    except Exception as e:
        print('--> Erro na funcao \'cont_Instancias_Orquestra\' <--')
        funcoes.printError(e)
        return -1

In [68]:
def instances_Report_Orquestra(showFinishedTasks=True):
    '''
        Funcao
        ------
            Lista todas instâncias de solicitações que a pessoa relacionada ao token possui permissão de consultar de acordo com filtros.
        
        Parametros
        ----------
            showFinishedTasks: boolean (obrigatorio)
                definir se vai filtrar por instancias fechadas ou instancias abertas das solicitacoes.
        Retorno 
        -------
            Retorna um DataFrame(Pandas) se nao ocorrer erro.
            Se erro, retorna -1
    '''
    try:
        # OBRIGATORIO o filtro de startDateInterval
        f = {"startDateIntervalBegin": "2000-01-01T00:00:00",
                "startDateIntervalEnd": "2030-12-31T23:59:59",
                "showFinishedInstanceTasks": True, 
                "showPendingInstanceTasks": False,
                "active": True}
        # Escolhe se vai mostrar tarefas fechadas ou pendentes
        if showFinishedTasks == False:
            f['showFinishedInstanceTasks'] = False
            f['showPendingInstanceTasks'] = True
        # Token da API User
        t=['enb4iHROSDegvkiZv17Fxw%2BkQ7q7Zidu1PKalcpC4o4%3D']
        # Requisicao 
        lista = request_Orquestra(method='GET',urlReq=url,typeReq='/api/2/instances/report',tokens=t,filters=f)
        df = pd.DataFrame(lista)
        return df
    except Exception as e:
        print('--> Erro na funcao \'instances_Report_Orquestra\' <--')
        funcoes.printError(e)
        return -1

def fluxo_personalizado(dataFrame):
    '''
        Funcao
        ------
           Faz a contagem de clientes no fluxo personalizado.
        
        Retorno 
        -------
            int: Retorna a quantidade de clientes no fluxo personalizado.
            Se erro, retorna -1
    '''
    try:
        df = dataFrame
        # Retorno com a contagem
        return len(df.loc[df['requestName'].str.lower().str.contains("normal")]) # calculando o fluxo normal
    except Exception as e:
        print('--> Erro na funcao \'fluxo_personalizado\' <--')
        funcoes.printError(e)
        return -1

def main_Func():
    # Requisicao com todos os dados iniciais
    lista = request_Orquestra(method=metodo,urlReq=url,typeReq=tipoUrl,tokens=tks)
    df = formata_DataFrame(lista)

    # CS1 Abertos sao os Efetivos Em espera + Em Aberto nas atribuicoes de cada usuario -> Pegar o relogio de 7/15 dias
    cs1 = df.loc[(df['requestName'].str.lower().str.replace(" ","").str.contains('projetov.20')) & ((df['taskName'].str.lower().str.contains('ligação do cs')) | (df['taskName'].str.lower().str.replace(" ","").str.contains('cooldown')))]
    contCs1Abertos = len(cs1)
    # CS1 Efetivos sao os Em espera do 'Servico Orquestra' -> Entrou no relogio de 7 ou 15 dias na semana do script, conta como efetivo
    cs1Efetivos = df.loc[(df['requestName'].str.lower().str.replace(" ","").str.contains('projetov.20')) 
                        & (df['taskName'].str.lower().str.replace(" ","").str.contains('cooldown'))
                        & (df['startDateTime'].between(periodo1,periodo2, inclusive='both'))]
    contCs1Efetivos = len(cs1Efetivos)
    # CS1 Atrasados
    contCS1Atrasadas = len(cs1.loc[(df['taskAtrasada'] == True) & (df['taskName'].str.lower().str.contains('ligação do cs'))])

    # CS2 Em aberto
    # Para degustacao a etapa -> 'Inicio do mes seguinte' contabiliza os em aberto etapa apos email- acomp personalizado 1/3
    cs2 = df.loc[(df['requestName'].str.lower().str.contains('degustação')) | (df['requestName'].str.lower().str.contains('normal')) & ~(df['taskName'].str.lower().str.contains('relatório')) | (df['taskName'].str.lower().str.contains('suporte'))]
    contCs2Abertos = len(cs2)
    # CS2 Efetivos - Filtrar pela semana de execucao do script e data de abertura da tarefa
    # Necessario contabilizar a quarta ligacao como efetivo
    cs2Efetivo = cs2.loc[~(cs2['taskName'].str.lower().str.contains('ligação do cs'))]
    contCs2Efetivos = len(cs2Efetivo)
    # CS2 Atrasados - ajustar datas
    cs2Atrasados = cs2.loc[(cs2['taskName'].str.lower().str.contains('ligação do cs')) & (cs2['taskAtrasada'] == True)]
    contCs2Atrasados = len(cs2Atrasados)

    # ----------- Report de instancias -----------
    instancesReport = instances_Report_Orquestra() # Listar todas instâncias de solicitações que a pessoa relacionada ao token possui permissão de consultar de acordo com filtros 
    # -------------------------------------------

    # Acompanhamento Personalizado
    contClientesfluxoNormal = fluxo_personalizado(instancesReport) 

    # Envio de Layout Pendentes 
    envLayout = df.loc[(df['requestName'] == 'Projeto  v. 20') & (df['taskName'] == 'Baixar Layout e apresentar ao cliente')]
    contEnvLayoutPendentes = len(envLayout)
    # Envio de Layout Realizados - Contagem por Períodos
    contEnvLayoutRealizados = cont_Instancias_Orquestra('projeto v.20','baixar layout',per1=periodo1,per2=periodo2,dataFrame=instancesReport)
    # Envio de Layout Atrasados
    contEnvLayoutAtrasadas = len(envLayout[(envLayout['taskName'] == 'Baixar Layout e apresentar ao cliente') & (envLayout['taskAtrasada'] == True)])

    # Envio de Documentos Pendentes
    envDoc = df.loc[(df['requestName'] == 'Projeto  v. 20') & (df['taskName'] == 'Entregar Documentação ao Cliente')]
    contEnvDocPendentes = len(envDoc)
    # Envio de Documentos Realizados - Contagem por Períodos
    contEnvDocRealizados = cont_Instancias_Orquestra('projeto v.20','entregar documentação', periodo1, periodo2, instancesReport)
    # Envio de Documentos Atrasados 
    contEnvDocAtrasados = len(envDoc.loc[envDoc['taskAtrasada'] == True])

    # Print
    print('\n-------------------------------------------------------------------------------------------------------------')
    print("Contatos Em Aberto CS1 - {0}\nContatos Efetivos CS1 - {1} <- No periodo de {3} e {4}.\nContatos Em Atraso CS1 - {2}".format(contCs1Abertos, contCs1Efetivos, contCS1Atrasadas, str(periodo1), str(periodo2)))
    print('Contatos Em Aberto CS2 - {0}\nContatos Efetivos CS2 - {1}\nContatos Em Atraso CS2 - {2}'.format(contCs2Abertos, contCs2Efetivos,contCs2Atrasados))
    print('Acompanhamento Personalizado - {0} <- No periodo de {1} e {2}'.format(contClientesfluxoNormal, str(periodo1), str(periodo2))) 
    print('Envio de Layouts Pendentes  - {0}\nEnvio de Layouts Realizados - {1} <- No periodo de {3} e {4}\nEnvio de Layouts Atrasados  - {2}'.format(contEnvLayoutPendentes, contEnvLayoutRealizados, contEnvLayoutAtrasadas, str(periodo1), str(periodo2)))
    print('Envio de Documentos Pendentes - {0}\nEnvio de Documentos Realizados  - {1} <- No periodo de {3} e {4}\nEnvio de Documentos Atrasados - {2}'.format(contEnvDocPendentes, contEnvDocRealizados, contEnvDocAtrasados, str(periodo1), str(periodo2))) 
    print('---------------------------------------------------------------------------------------------------------------')

In [59]:
lista = request_Orquestra(method=metodo,urlReq=url,typeReq=tipoUrl,tokens=tks)
df = formata_DataFrame(lista)

---> Tipo de requisicao: 'GET'
---> Tipo de requisicao: 'GET'
[]
---> Tipo de requisicao: 'GET'
---> Tipo de requisicao: 'GET'
[]
---> Tipo de requisicao: 'GET'
---> Tipo de requisicao: 'GET'
[]
---> Tipo de requisicao: 'GET'
---> Tipo de requisicao: 'GET'
[]
---> Tipo de requisicao: 'GET'
---> Tipo de requisicao: 'GET'
---> Tipo de requisicao: 'GET'
[]
Contagem de tarefas "ANTES" da remocao das tarefas duplicatas: 214
Contagem de tarefas "DEPOIS" da remocao das tarefas duplicatas: 214


C:\Users\Lukas\AppData\Local\Temp\ipykernel_12520\419362762.py:22: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:,'expirationDateTime'] = df.loc[:,'expirationDateTime'].dt.strftime('%Y-%m-%d')
C:\Users\Lukas\AppData\Local\Temp\ipykernel_12520\419362762.py:23: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:,'startDateTime'] = df.loc[:,'startDateTime'].dt.strftime('%Y-%m-%d')


In [63]:
# CS1 Abertos sao os Efetivos Em espera + Em Aberto nas atribuicoes de cada usuario -> Pegar o relogio de 7/15 dias
cs1 = df.loc[(df['requestName'].str.lower().str.replace(" ","").str.contains('projetov.20')) & ((df['taskName'].str.lower().str.contains('ligação do cs')) | (df['taskName'].str.lower().str.replace(" ","").str.contains('cooldown')))]
contCs1Abertos = len(cs1)

# CS1 Efetivos sao os Em espera do 'Servico Orquestra'
# Entrou no relogio de 7 ou 15 dias na semana do script, conta como efetivo
cs1Efetivos = df.loc[(df['requestName'].str.lower().str.replace(" ","").str.contains('projetov.20')) 
                    & (df['taskName'].str.lower().str.replace(" ","").str.contains('cooldown'))
                    & (df['startDateTime'].between(periodo1,periodo2, inclusive='both'))]
contCs1Efetivos = len(cs1Efetivos)

# CS1 Atrasados 
contCS1Atrasadas = len(cs1.loc[(df['taskAtrasada'] == True) & (df['taskName'].str.lower().str.contains('ligação do cs'))])
print("Contatos Em Aberto CS1 - {0}\nContatos Efetivos CS1 - {1} <- No período de {3} e {4}.\nContatos Em Atraso CS1 - {2}".format(contCs1Abertos,contCs1Efetivos,contCS1Atrasadas,str(periodo1), str(periodo2)))

Contatos Em Aberto CS1 - 68
Contatos Efetivos CS1 - 37 <- No período de 2022-10-28 e 2022-11-04.
Contatos Em Atraso CS1 - 21


C:\Users\Lukas\AppData\Local\Temp\ipykernel_12520\3886294541.py:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len(cs1[cs1['taskName'].str.lower().str.contains('ligação do cs') & (df['taskAtrasada'] == False)])


0

In [69]:
# ----------- Report de instancias -----------
instancesReport = instances_Report_Orquestra() # Listar todas instâncias de solicitações que a pessoa relacionada ao token possui permissão de consultar de acordo com filtros 
# -------------------------------------------

---- >> Requisicao lenta... << ---- 
---> Tipo de requisicao: 'GET'
---> Tipo de requisicao: 'GET'
---> Tipo de requisicao: 'GET'
---> Tipo de requisicao: 'GET'
---> Tipo de requisicao: 'GET'
---> Tipo de requisicao: 'GET'
---> Tipo de requisicao: 'GET'
---> Tipo de requisicao: 'GET'
---> Tipo de requisicao: 'GET'
---> Tipo de requisicao: 'GET'
---> Tipo de requisicao: 'GET'
---> Tipo de requisicao: 'GET'
---> Tipo de requisicao: 'GET'
---> Tipo de requisicao: 'GET'
---> Tipo de requisicao: 'GET'
---> Tipo de requisicao: 'GET'
---> Tipo de requisicao: 'GET'
---> Tipo de requisicao: 'GET'
---> Tipo de requisicao: 'GET'
---> Tipo de requisicao: 'GET'
---> Tipo de requisicao: 'GET'
---> Tipo de requisicao: 'GET'
---> Tipo de requisicao: 'GET'
---> Tipo de requisicao: 'GET'
---> Tipo de requisicao: 'GET'
---> Tipo de requisicao: 'GET'
---> Tipo de requisicao: 'GET'
[]


In [70]:
contEnvLayoutRealizados = cont_Instancias_Orquestra('projeto v.20','baixar layout',per1=periodo1,per2=periodo2,dataFrame=instancesReport)
print(contEnvLayoutRealizados)

4


In [ ]:
#instanceTasksDF = pd.DataFrame(proj.loc[proj['instanceTasks'].str.len() != 0]['instanceTasks'].explode().to_list())
#instanceTasksDF
#taskDF = pd.DataFrame(instanceTasksDF['task'].to_list())
#taskDF.loc[taskDF['name'].str.lower().str.contains('baixar layout')] # idtask -> 14417